In [1]:
import numpy as np
import math
from keras import optimizers
from utils import *
from model import *
import random

2022-07-04 21:41:48.417300: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-04 21:41:48.474663: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 21:41:48.474689: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Load Data

In [2]:
SBU_dir = "data/SBU"
dataset = GETDATA(SBU_dir)
train, test = dataset.get_data()

loading data from: data/SBU


# How to use the Model

In models.py, there is a implementation of the model (multi_person).

First, you need to pass the number of max bodies in a frame of your dataset. Example for SBU kinect Dataset, max number of bodies is 2. So while initializing model, pass the number of max_bodies

Ex: 
1. multi_person(2) if max bodies is 2
2. multi_person(20) if max bodies is 20

You can also pass other parameters like: number of frames, number of joints, dimensions of the joints

Ex: **multi_person(5, frame_l=32, joint_n=25, joint_d=3)**: There are maximum 5 bodies, 32 frames, 25 joints and joint is of 3 dimension.

By default, frame_l=16, joint_n=15, joint_d=3.


# How to pass input to the model
You need to pass a list of input to the model:

($S_1$, $M_1$, $S_2$, $M_2$, $S_3$, $M_3$, ..., $S_n$, $M_n$ )

where:

$S_i$ is the skeletal posture of person $i$

$M_i$ is the temporal difference of person $i$

$n$ is the maximum number of bodies


**If the number of bodies in current frame is less than n, then rest of the skeletal posture and temporal difference should be a matrix of 0's**

Dimensions of $S_i$ and $M_i$ are **(frame_l * joint_n * joint_d)**


You may need to train the model multiple times to achieve best accuracy.

# Initialize the model

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
model = multi_person(2)
model.compile(optimizer='adam', loss='mse', metrics = ['accuracy'])
model.summary()

2022-07-04 21:41:51.120584: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-04 21:41:51.120606: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-04 21:41:51.120622: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (joey): /proc/driver/nvidia/version does not exist
2022-07-04 21:41:51.121025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inp_j_0 (InputLayer)           [(None, 16, 15, 3)]  0           []                               
                                                                                                  
 inp_j_diff_0 (InputLayer)      [(None, 16, 15, 3)]  0           []                               
                                                                                                  
 inp_j_1 (InputLayer)           [(None, 16, 15, 3)]  0           []                               
                                                                                                  
 inp_j_diff_1 (InputLayer)      [(None, 16, 15, 3)]  0           []                               
                                                                                            

# Train the model

In [5]:
EPOCHS = 50
for e in range(EPOCHS):

    X_0, X_1, X_2, X_3 = [], [], [], []
    Y = []

    for i in range(1,9):                 # loop all the 8 classes
        for j in range(len(train[i])):   # loop all samples within the ith class
            

            # First person pose
            s_1 = get_person_pose(train[i][j].T[:,:45])
            m_1 = temporal_difference(s_1)

            # Second person pose
            s_2 = get_person_pose(train[i][j].T[:,45:])
            m_2 = temporal_difference(s_2)

            # Mirror Augmentation
            s_1_m, s_2_m = mirror(s_1,s_2)
            m_1_m = temporal_difference(s_1_m)
            m_2_m = temporal_difference(s_2_m)

            # label
            label = np.zeros(8)
            label[i-1] = 1


            # Append all postures
            X_0.append(s_1)
            X_1.append(m_1)
            X_2.append(s_2)
            X_3.append(m_2)
            
            Y.append(label)


            # Append all mirrored postures
            X_0.append(s_1_m)
            X_1.append(m_1_m)
            X_2.append(s_2_m)
            X_3.append(m_2_m)
            
            Y.append(label)

        


    X_0 = np.stack(X_0)
    X_1 = np.stack(X_1)
    X_2 = np.stack(X_2)
    X_3 = np.stack(X_3)
    Y = np.stack(Y)
    
    
    history = model.fit([X_0,X_1,X_2,X_3], Y, batch_size=32, epochs=1, verbose=True, shuffle=True)


14/14 [==============================] - 1s 46ms/step - loss: 0.0015 - accuracy: 0.9977


# Test model

In [6]:
import keras

if input("Do you want to load any prev model? (y/n) ") == "y":
    load_name = input("Name of the model: ")
    model = keras.models.load_model(f"all_model/{load_name}")

print("\n")
    
X_TEST_0 = []
X_TEST_1 = []
X_TEST_2 = []
X_TEST_3 = []
Y_TEST = []

for i in range(1,9):
    for j in range(len(test[i])):

        s_1 = get_person_pose(test[i][j].T[:,:45])
        s_2 = get_person_pose(test[i][j].T[:,45:])
        
        m_1 = temporal_difference(s_1)
        m_2 = temporal_difference(s_2)
     
        X_TEST_0.append(s_1)
        X_TEST_1.append(m_1)
        X_TEST_2.append(s_2)
        X_TEST_3.append(m_2)
        
        label = np.zeros(8)
        label[i-1] = 1
        Y_TEST.append(label)

X_TEST_0 = np.stack(X_TEST_0)
X_TEST_1 = np.stack(X_TEST_1)
X_TEST_2 = np.stack(X_TEST_2)
X_TEST_3 = np.stack(X_TEST_3)
X_TEST = [X_TEST_0,X_TEST_1,X_TEST_2,X_TEST_3]
Y_TEST = np.stack(Y_TEST)


from sklearn.metrics import classification_report

y_pred = model.predict(X_TEST)

Y_TEST = np.argmax(Y_TEST, axis=1)
y_pred = np.argmax(y_pred, axis=1)


print(classification_report(Y_TEST, y_pred))

Do you want to load any prev model? (y/n) n


3/3 [==============================] - 0s 19ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.65      1.00      0.79        11
           2       1.00      1.00      1.00        10
           3       0.88      0.88      0.88         8
           4       1.00      0.67      0.80         3
           5       1.00      1.00      1.00         5
           6       1.00      0.80      0.89        10
           7       1.00      0.62      0.77         8

    accuracy                           0.89        66
   macro avg       0.94      0.87      0.89        66
weighted avg       0.93      0.89      0.90        66



# Save the model

In [ ]:
if input("Do you want to save the model? (y/n) ") == "y":
    name = input("Name of the Model: ")
    model.save(f"all_models/{name}")

# Visulize the mirror augmentation

In [ ]:
i = 10
draw_2d_pose([p_0[i],p_1[i]])
print(p_0[i].shape)
p_0_new, p_1_new = mirror(p_0,p_1)
draw_2d_pose([p_0_new[i],p_1_new[i]])